In [1]:
import time, re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
def initialize_driver(executable_path):

    s = Service(executable_path=executable_path)
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    return driver

In [3]:
def scrape_reviews(driver, url):

    driver.get(url)
    time.sleep(5)

    reviews_button = driver.find_element(By.CSS_SELECTOR,'#reviews-medley-footer > div.a-row.a-spacing-medium > a')
    reviews_button.get_attribute('href')
    reviews_button.click()
    time.sleep(0.75)

    totals =  driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[4]/div").text
    total_ratings_match = re.search(r'([\d,]+) total ratings', totals)
    if total_ratings_match:

        total_ratings = int(total_ratings_match.group(1).replace(",", ""))  # Remove comma

        # Extract total reviews
        total_reviews_match = re.search(r'([\d,]+) with reviews', totals)
        if total_reviews_match:
            total_reviews = int(total_reviews_match.group(1).replace(",", ""))  # Remove comma

    n = round(total_reviews/10)
    print(n)
    star_rating = []
    review = []
    name = []
    review_title = []
    votes = []
    verified = []

    for j in range(n):
        review_elements = driver.find_elements(By.CSS_SELECTOR,'i[data-hook="review-star-rating"] span.a-icon-alt')
        page_total_reviews = len(review_elements)

        for i in range(1, page_total_reviews + 1):
            try:
                rating = driver.find_elements(By.CSS_SELECTOR,'i[data-hook="review-star-rating"] span.a-icon-alt')
                rating = rating[i-1].get_attribute('innerHTML')
                star = float(rating.split()[0])

                rev = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[4]/span/span").text
                rev_name = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[1]/a/div[2]/span").text
                title = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[2]/a/span[2]").text
                
                try: helpful_votes =  driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[5]/div/span[1]/div[1]/span").text
                except: helpful_votes = '0 helpful votes'

                try: verified_purchase = driver.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i+1}]/div/div/div[3]/a[2]/span").text
                except: verified_purchase = 'Not a Verified Purchase'

            except Exception as e: print(e)

            star_rating.append(star)
            review.append(rev)
            name.append(rev_name)
            review_title.append(title)
            if helpful_votes == 'Helpful': votes.append('0 helpful votes')
            else: votes.append(helpful_votes)
            verified.append(verified_purchase)
        
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#cm_cr-pagination_bar > ul > li.a-last > a'))
            )
            next_button.click()
            time.sleep(1.5)
        except:
            break
        print(f'{j} is done')
    return star_rating, review, name, review_title, votes, verified

In [4]:
def main():
    urls = ['https://www.amazon.com/dp/B0BYLJ3HH7?ref_=cm_sw_r_cp_ud_dp_K9518ZWP7QPY9CGBKXD3&th=1', 'https://www.amazon.com/dp/B08M91FTJW?_encoding=UTF8&psc=1&ref_=cm_sw_r_cp_ud_dp_D4PC0KH8VM0MZ9RWQM8C',
            'https://www.amazon.com/dp/B08XYP6BJV?ref_=cm_sw_r_cp_ud_dp_FX17EPC24AF0YC1B2JX4_1', 'https://www.amazon.com/dp/B09WZF9633?ref_=cm_sw_r_cp_ud_dp_4T7X35DYN1G52PX889BN_1&th=1',
            'https://www.amazon.com/dp/B07F3SNQT5?ref_=cm_sw_r_cp_ud_dp_YAM85QGN8A18TXN92KWV&th=1']

    driver = initialize_driver('C:/Users/uttar/OneDrive/Desktop/Data Mining/chromedriver.exe')

    all_review_dfs = []

    for url in urls:
        star_rating, review, name, review_title, votes, verified = scrape_reviews(driver, url)
        review_df = pd.DataFrame({
            'User_Star_rating': star_rating,
            'Review_content': review,
            'Reviewer_name': name,
            'Review_title': review_title,
            'Helpful_votes': votes,
            'Verified_purchase': verified
        })

        all_review_dfs.append(review_df)

    driver.quit()

    return all_review_dfs

In [5]:
# Call the main function to get data for all URLs
all_review_dfs = main()

10
0 is done
1 is done
2 is done
3 is done
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[6]/div/div/div[4]/span/span"}
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60757AD22+56930]
	(No symbol) [0x00007FF6074EF622]
	(No symbol) [0x00007FF6073A42E5]
	(No symbol) [0x00007FF6073E98ED]
	(No symbol) [0x00007FF6073E9A2C]
	(No symbol) [0x00007FF60742A967]
	(No symbol) [0x00007FF60740BCDF]
	(No symbol) [0x00007FF6074281E2]
	(No symbol) [0x00007FF60740BA43]
	(No symbol) [0x00007FF6073DD438]
	(No symbol) [0x00007FF6073DE4D1]
	GetHandleVerifier [0x00007FF6078F6AAD+3709933]
	GetHandleVerifier [0x00007FF60794FFED+4075821]
	GetHandleVerifier [0x00007FF60794817F+4043455]
	GetHandleVerifier [0x00007FF607619756+706710]
	

In [ ]:
all_review_dfs

[    User_Star_rating                                     Review_content  \
 0                5.0  It was packaged very well, Amazon packaging, n...   
 1                5.0  I had an older android and was having a hard t...   
 2                5.0  The phone came in perfect conditions, without ...   
 3                1.0  If I could send negative stars I would. The ph...   
 4                4.0  Excelente equipo la verdad me sorprendido más ...   
 ..               ...                                                ...   
 90               2.0  This company is a joke. You have a problem wit...   
 91               5.0  Got this as a gift for my girlfriend. She had ...   
 92               1.0  Wasn’t able to use or set up iPhone due to it ...   
 93               1.0  Bought this thinking great, an unlocked iPhone...   
 94               5.0  Tiene el 100% de la bateria y esta en excelent...   
 
                                     Reviewer_name  \
 0                              

In [ ]:
for i, df in enumerate(all_review_dfs):
    df.to_excel(f'../data/amazon_url_{i+1}_review_data.xlsx', index=False)

In [ ]:
def concatenate_excel_files(file_names):
    dfs = []
    for file in file_names:
        df = pd.read_excel(file)
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)
    return concatenated_df

file_names_r = ["amazon_url_1_review_data.xlsx", "amazon_url_2_review_data.xlsx", "amazon_url_3_review_data.xlsx", "amazon_url_4_review_data.xlsx", "amazon_url_5_review_data.xlsx"]

# Concatenate the Excel files
concatenated_df_r = concatenate_excel_files(file_names_r)

# Write the concatenated DataFrame to a new Excel file
concatenated_df_r.to_excel("../data/Review_data_all.xlsx", index=False)

print("Concatenation complete. New Excel file created.")

Concatenation complete. New Excel file created.
